## Overview

This analysis involves running the REopt API with two distinct scenarios:
- **Scenario 1: PV only** - Incorporates only a photovoltaic (PV) system.
- **Scenario 2: PV + BESS** - Includes both a PV system and a Battery Energy Storage System (BESS).

The API responses are stored as JSON files in the outputs folder for subsequent processing.


## Note:
Run this section of code first (before running the rest) and copy the Run UUIDs provided at the end as inputs within the main script as `run_uuid_1` and `run_uuid_2`

In [1]:
import pandas as pd
import json
import requests
import time
import os
import uuid
from collections import defaultdict
import re

API_KEY = "your_key_here"  # Replace with your API key

# following is not necessary but silences warnings:
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

inputs_path = os.path.join(".", 'inputs')
outputs_path = os.path.join(".", 'outputs')

# Define the post data for both scenarios
post_1 = {
    "Settings": {
      "run_bau": True
    },
    "Site": {
        "longitude": -118.1164613,
        "latitude": 34.5794343
    },
    "PV": {
        "array_type": 0
    },
    "ElectricLoad": {
        "doe_reference_name": "RetailStore",
        "annual_kwh": 100000.0,
        "year": 2017
    },
    "ElectricTariff": {
        "urdb_label": "5ed6c1a15457a3367add15ae"
    },
    "Financial": {
        "elec_cost_escalation_rate_fraction": 0.026,
        "owner_discount_rate_fraction": 0.081,
        "analysis_years": 20,
        "offtaker_tax_rate_fraction": 0.4,
        "owner_tax_rate_fraction": 0.4,
        "om_cost_escalation_rate_fraction": 0.025
    }
}

post_2 = {
    "Settings": {
        "run_bau": True
    },
    "Site": {
        "longitude": -118.1164613,
        "latitude": 34.5794343
    },
    "PV": {
        "array_type": 0
    },
    "ElectricLoad": {
        "doe_reference_name": "RetailStore",
        "annual_kwh": 100000.0,
        "year": 2017
    },
    "ElectricStorage":{
    },
    "ElectricTariff": {
        "urdb_label": "5ed6c1a15457a3367add15ae"
    },
    "Financial": {
        "elec_cost_escalation_rate_fraction": 0.026,
        "owner_discount_rate_fraction": 0.081,
        "analysis_years": 20,
        "offtaker_tax_rate_fraction": 0.4,
        "owner_tax_rate_fraction": 0.4,
        "om_cost_escalation_rate_fraction": 0.025
    }
}

# Function to POST inputs to the API
def post_inputs_to_api(post_data, api_key, api_url):
    headers = {'x-api-key': api_key}
    response = requests.post(f"{api_url}/job/", headers=headers, json=post_data, verify=False)
    response.raise_for_status()
    return response.json()["run_uuid"]

# Function to poll for results
def poll_for_results(run_uuid, api_key, api_url, interval=5):
    headers = {'x-api-key': api_key}
    while True:
        response = requests.get(f"{api_url}/job/{run_uuid}/results/", headers=headers, verify=False)
        response.raise_for_status()
        result = response.json()
        if result["status"] != "Optimizing...":
            return result
        time.sleep(interval)

# Define the API key and URL
root_url = "https://developer.nrel.gov/api/reopt/stable"

# Post the inputs to the API and get the run_uuid for both scenarios
run_uuid_1 = post_inputs_to_api(post_1, API_KEY, root_url)
print(f"Run UUID for Scenario 1: {run_uuid_1}")

run_uuid_2 = post_inputs_to_api(post_2, API_KEY, root_url)
print(f"Run UUID for Scenario 2: {run_uuid_2}")

# Poll for results for both scenarios
results_1 = poll_for_results(run_uuid_1, API_KEY, root_url)
print("Results retrieved successfully for Scenario 1.")

results_2 = poll_for_results(run_uuid_2, API_KEY, root_url)
print("Results retrieved successfully for Scenario 2.")

# Save the results to JSON files
outputs_file_name_1 = f"response_{run_uuid_1}.json"
with open(os.path.join(outputs_path,outputs_file_name_1), 'w') as fp:
    json.dump(results_1, fp, indent=4)
print(f"Results saved to {outputs_file_name_1}")

outputs_file_name_2 = f"response_{run_uuid_2}.json"
with open(os.path.join(outputs_path,outputs_file_name_2), 'w') as fp:
    json.dump(results_2, fp, indent=4)
print(f"Results saved to {outputs_file_name_2}")


Run UUID for Scenario 1: b620d505-3891-47e4-9be2-b7f150a6dd82
Run UUID for Scenario 2: 9e7b3977-f3a0-4d2b-b371-392e34b34c50
Results retrieved successfully for Scenario 1.
Results retrieved successfully for Scenario 2.
Results saved to response_b620d505-3891-47e4-9be2-b7f150a6dd82.json
Results saved to response_9e7b3977-f3a0-4d2b-b371-392e34b34c50.json


## Key Functions and Steps in the Analysis

### 1. `get_scenario_results`
Retrieves the results from the REopt API for a specified scenario using its unique run UUID. This function:
- Sends a GET request to the REopt API.
- Fetches the results in JSON format.

### 2. `process_scenarios`
Processes multiple scenarios to generate a consolidated DataFrame. Key steps include:
- **Data Extraction**: Utilizes the `get_REopt_data` function with a specified `config` to extract relevant data points from the API responses.
- **BAU Data Handling**: Checks and manages Business-As-Usual (BAU) values to ensure data consistency across scenarios.
- **Final DataFrame Creation**: Combines data from all scenarios into a single DataFrame and processes it for final output.

### 3. `get_REopt_data`
Extracts specific data points from the API response JSON based on a configuration (`config`). This function:
- Flattens the nested JSON structure.
- Retrieves values according to the configuration settings.
- Cleans and structures the data into a pandas DataFrame.

### 4. `generate_data_dict`
Generates a data dictionary based on the provided configuration and flattened data. This dictionary is then converted into a DataFrame for further analysis.


In [2]:
def get_with_suffix(df, key, suffix, default_val=0):
    if not key.endswith("_bau"):
        key = f"{key}{suffix}"
    return df.get(key, default_val)

def flatten_dict(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def generate_data_dict(config, df_gen, suffix):
    data_dict = defaultdict(list)
    for var_key, col_name in config:
        if callable(var_key):
            val = var_key(df_gen)
        else:
            val = get_with_suffix(df_gen, var_key, suffix, "-")
        data_dict[col_name].append(val)
    return data_dict

def clean_data_dict(data_dict):
    for key, value_array in data_dict.items():
        new_value_array = ["-" if v in [0, float("nan"), "NaN", "0", "0.0", "$0.0", -0, "-0", "-0.0", "-$0.0", None] else v for v in value_array]
        data_dict[key] = new_value_array
    return data_dict

def remove_empty_data_columns(df):
    # Remove columns where all the entries are "-"
    df_cleaned = df.loc[:, (df != '-').any(axis=0)]
    return df_cleaned

def get_REopt_data(data_f, scenario_name, config, cur_gen_size=0):
    scenario_name_str = str(scenario_name)
    suffix = "_bau" if re.search(r"(?i)\bBAU\b", scenario_name_str) else ""
    
    df_gen = flatten_dict(data_f)
    data_dict = generate_data_dict(config, df_gen, suffix)
    data_dict = clean_data_dict(data_dict)
    data_dict["Scenario"] = [scenario_name_str]

    col_order = ["Scenario"] + [col_name for _, col_name in config]
    df_res = pd.DataFrame(data_dict)
    df_res = df_res[col_order]

    return df_res

def get_bau_values(mock_scenarios, config):
    bau_values = {col_name: None for _, col_name in config}
    for scenario in mock_scenarios:
        df_gen = flatten_dict(scenario["outputs"])
        for var_key, col_name in config:
            key = var_key.__code__.co_consts[1]
            key_bau = f"{key}_bau"
            if key_bau in df_gen:
                value = df_gen[key_bau]
                if bau_values[col_name] is None:
                    bau_values[col_name] = value
                elif bau_values[col_name] != value:
                    raise ValueError(f"Inconsistent BAU values for {col_name}. This should only be used for portfolio cases with the same Site, ElectricLoad, and ElectricTariff for energy consumption and energy costs.")
    return bau_values

def get_scenario_results(run_uuid):
    results_url = f"{root_url}/job/{run_uuid}/results/?api_key={API_KEY}"
    response = requests.get(results_url, verify=False)
    response.raise_for_status()
    return response.json()

def sum_numeric(vector, default_value=0.0):
    numeric_vector = [(x if isinstance(x, (int, float)) and not pd.isna(x) else default_value) for x in vector]
    return sum(numeric_vector)

def process_scenarios(scenarios):
    config = [
        (lambda df: get_with_suffix(df, "PV.size_kw", ""), "PV Size (kW)"),
        (lambda df: get_with_suffix(df, "ElectricStorage.size_kw", ""), "Battery Size (kW)"),
        (lambda df: get_with_suffix(df, "ElectricStorage.size_kwh", ""), "Battery Capacity (kWh)"),
        # (lambda df: get_with_suffix(df, "Wind.size_kw", ""), "Wind Size (kW)"),
        # (lambda df: get_with_suffix(df, "CHP.size_kw", ""), "CHP Size (kW)"),
        (lambda df: get_with_suffix(df, "PV.annual_energy_produced_kwh", ""), "PV Total Electricity Produced (kWh)"),
        # (lambda df: get_with_suffix(df, "Wind.annual_energy_produced_kwh", ""), "Wind Total Electricity Produced (kWh)"),
        # (lambda df: get_with_suffix(df, "CHP.annual_electric_energy_produced_kwh", ""), "CHP Total Electricity Produced (kWh)"),
        (lambda df: get_with_suffix(df, "ElectricUtility.annual_energy_supplied_kwh", ""), "Grid Purchased Electricity (kWh)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_energy_cost_before_tax", ""), "Electricity Energy Cost ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_demand_cost_before_tax", ""), "Electricity Demand Cost ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_fixed_cost_before_tax", ""), "Utility Fixed Cost($)"),
        # (lambda df: get_with_suffix(df, "ElectricTariff.year_one_export_benefit_before_tax", ""), "Electricity Export Benefit ($)"),
        # (lambda df: get_with_suffix(df, "Boiler.fuel_used_mmbtu", ""), "Boiler Fuel (MMBtu)"),
        # (lambda df: get_with_suffix(df, "CHP.fuel_used_mmbtu", ""), "CHP Fuel (MMBtu)"),
        # (lambda df: get_with_suffix(df, "Boiler.annual_thermal_production_mmbtu", ""), "Boiler Thermal Production (MMBtu)"),
        # (lambda df: get_with_suffix(df, "CHP.annual_thermal_production_mmbtu", ""), "CHP Thermal Production (MMBtu)"),
        # (lambda df: get_with_suffix(df, "Financial.om_cost_escalation_rate_fraction", ""), "O&M Cost Increase ($)"),
        (lambda df: get_with_suffix(df, "Financial.simple_payback_years", ""), "Payback Period (years)"),
        (lambda df: get_with_suffix(df, "Financial.lifecycle_capital_costs", ""), "Gross Capital Cost ($)"),
        # (lambda df: get_with_suffix(df, "Financial.total_incentives", ""), "Federal Tax Incentive (30%)"),
        # (lambda df: get_with_suffix(df, "Financial.total_utility_annual_costs", ""), "IAC Grant ($)"),
        (lambda df: get_with_suffix(df, "Site.annual_emissions_tonnes_CO2", ""), "CO2 Emissions (tonnes)"),
        (lambda df: round(100 * (get_with_suffix(df, "ElectricTariff.year_one_bill_before_tax_bau", "") - 
                            get_with_suffix(df, "ElectricTariff.year_one_bill_before_tax", "")) / 
                            (get_with_suffix(df, "ElectricTariff.year_one_bill_before_tax_bau", 1) + 1e-6)
                            ), "Year 1 Utility Savings (%)"),
        (lambda df: round(100 * get_with_suffix(df, "Site.renewable_electricity_fraction", "")), "RE Penetration (%)"),
        (lambda df: get_with_suffix(df, "Site.annual_emissions_tonnes_CO2", ""), "Annual CO2 Emissions (Tons)"),
        (lambda df: get_with_suffix(df, "Site.lifecycle_emissions_tonnes_CO2", ""), "Lifecycle CO2 Emissions (Tons)"),
        (lambda df: round(100 * get_with_suffix(df, "Site.lifecycle_emissions_reduction_CO2_fraction", "")), "Lifecycle CO2 Reduction (%)"),
        (lambda df: get_with_suffix(df, "ElectricUtility.annual_energy_supplied_kwh", ""), "Year 1 Electric Grid Purchases (kWh)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_energy_cost_before_tax", ""), "Year 1 Energy Charges ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_demand_cost_before_tax", ""), "Year 1 Demand Charges ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_fixed_cost_before_tax", ""), "Year 1 Fixed Cost Charges ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_coincident_peak_cost_before_tax", ""), "Year 1 Coincident Peak Charges ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_bill_before_tax", ""), "Year 1 Total Electric Bill Costs ($)"),
        (lambda df: get_with_suffix(df, "CHP.year_one_fuel_cost_before_tax", ""), "Year 1 CHP Fuel Cost ($)"),
        (lambda df: get_with_suffix(df, "ExistingBoiler.year_one_fuel_cost_before_tax", ""), "Year 1 Existing Boiler Fuel Cost ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_energy_cost_before_tax_bau", "") - get_with_suffix(df, "ElectricTariff.year_one_energy_cost_before_tax", ""), "Year 1 Energy Charge Savings ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_demand_cost_before_tax_bau", "") - get_with_suffix(df, "ElectricTariff.year_one_demand_cost_before_tax", ""), "Year 1 Demand Charge Savings ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_bill_before_tax_bau", "") - get_with_suffix(df, "ElectricTariff.year_one_bill_before_tax", ""), "Year 1 Total Electric Bill Savings ($)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.lifecycle_fixed_cost_after_tax", "") + get_with_suffix(df, "ElectricTariff.lifecycle_demand_cost_after_tax", "") + get_with_suffix(df, "ElectricTariff.lifecycle_energy_cost_after_tax", ""), "Lifecycle Utility Electricity Cost ($)"),
        (lambda df: get_with_suffix(df, "Financial.lcc", ""), "Total Lifecycle Cost ($)"),
        (lambda df: get_with_suffix(df, "Financial.npv", ""), "Net Present Value ($)"),
        (lambda df: round(100 * (get_with_suffix(df, "Financial.npv", "")) / (get_with_suffix(df, "Financial.lcc_bau", 1, "") + 1e-6)), "Savings Compared to BAU (%)"),
        (lambda df: get_with_suffix(df, "PV.annual_energy_produced_kwh", ""), "Annual PV Production (kWh)"),
        (lambda df: sum_numeric(get_with_suffix(df, "PV.electric_curtailed_series_kw", "", [])), "PV Curtailed (kWh)"),
        (lambda df: get_with_suffix(df, "PV.lcoe_per_kwh", ""), "PV Levelized Cost of Energy ($/kWh)"),
        (lambda df: get_with_suffix(df, "ElectricTariff.year_one_export_benefit_before_tax", ""), "Year 1 Net Metering Benefit ($)"),
        (lambda df: get_with_suffix(df, "Financial.annualized_payment_to_third_party", ""), "Annual Payment to Third-Party ($)"),
        (lambda df: round(get_with_suffix(df, "Financial.annualized_payment_to_third_party", "") / 12), "Monthly Payment to Third-Party ($)")
    ]

    bau_values = get_bau_values(scenarios, config)

    combined_df = pd.DataFrame()
    for scenario in scenarios:
        run_uuid = scenario['run_uuid']
        df_result = get_REopt_data(scenario["outputs"], run_uuid, config)
        df_result = df_result.set_index('Scenario').T
        df_result.columns = [run_uuid]
        if combined_df.empty:
            combined_df = df_result
        else:
            combined_df = combined_df.join(df_result, how='outer')

    bau_data = {key: [value] for key, value in bau_values.items()}
    bau_data["Scenario"] = ["BAU"]
    df_bau = pd.DataFrame(bau_data)

    combined_df = pd.concat([df_bau, combined_df.T]).reset_index(drop=True)
    combined_df = clean_data_dict(combined_df.to_dict(orient="list"))
    combined_df = pd.DataFrame(combined_df)

    combined_df = combined_df[["Scenario"] + [col for col in combined_df.columns if col != "Scenario"]]

    return combined_df

def summary_by_runuuids(run_uuids):
    if len(run_uuids) == 0:
        return {'Error': 'Must provide one or more run_uuids'}

    for r_uuid in run_uuids:
        if type(r_uuid) != str:
            return {'Error': 'Provided run_uuids type error, must be string. ' + str(r_uuid)}
        
        try:
            uuid.UUID(r_uuid)
        except ValueError as e:
            if e.args[0] == "badly formed hexadecimal UUID string":
                return {"Error": str(e)}
            else:
                return {"Error": str(e)}
    
    try:
        scenarios = []
        for run_uuid in run_uuids:
            results = get_scenario_results(run_uuid)
            scenarios.append(results)
        return {'scenarios': scenarios}
    except Exception as e:
        return {"Error": str(e)}

### Main Script Execution (`main`)

- **Retrieves the Scenario Data**: Retrieves results from the REopt API for each scenario.
- **Data Processing**: Processes the scenarios using the functions outlined above.
- **Output Handling**: Converts the processed data into JSON format, and provides options to save it as a CSV file.

In [3]:
def main():
    run_uuid_1 = "b620d505-3891-47e4-9be2-b7f150a6dd82"
    run_uuid_2 = "9e7b3977-f3a0-4d2b-b371-392e34b34c50"

    run_uuids = [run_uuid_1, run_uuid_2]

    scenarios = summary_by_runuuids(run_uuids)
    if 'scenarios' not in scenarios:
        print(f"Error: {scenarios['Error']}")
        return

    final_df = process_scenarios(scenarios['scenarios'])
    final_df = remove_empty_data_columns(final_df)

    final_df.iloc[1:, 0] = run_uuids
    
    display(final_df)
    
    # Convert the final DataFrame to JSON format
    json_data = final_df.reset_index().to_dict(orient='records')
    
    with open('results.json', 'w') as json_file:
        json.dump(json_data, json_file, indent=2)
        
    json_string = json.dumps(json_data, indent=2)
    print(json_string)
    
    # Convert the JSON string back to a DataFrame for saving as CSV
    json_df = pd.read_json(json_string)
    json_df = json_df.T
    
    json_df.to_csv('transposed_results.csv')

main()

,Scenario,PV Size (kW),Battery Size (kW),Battery Capacity (kWh),PV Total Electricity Produced (kWh),Grid Purchased Electricity (kWh),Electricity Energy Cost ($),Electricity Demand Cost ($),Utility Fixed Cost($),Payback Period (years),...,Year 1 Energy Charge Savings ($),Year 1 Demand Charge Savings ($),Year 1 Total Electric Bill Savings ($),Lifecycle Utility Electricity Cost ($),Total Lifecycle Cost ($),Net Present Value ($),Savings Compared to BAU (%),Annual PV Production (kWh),PV Curtailed (kWh),PV Levelized Cost of Energy ($/kWh)
0,BAU,-,-,-,-,100000.00,9893.91,7689.50,1600.0,-,...,-,-,-,13416.92,160889.6584,-,-,-,-,-
1,b620d505-3891-47e4-9be2-b7f150a6dd82,32.8409,-,-,58151.95,47041.00,4587.02,6340.71,1600.0,5.42,...,5306.89,1348.79,6655.68,105068.25,134164.0247,26725.63,17,58151.95,5192.78,0.0448
2,9e7b3977-f3a0-4d2b-b371-392e34b34c50,45.2187,13.08,65.62,80069.28,26455.65,2441.63,2702.23,1600.0,5.9,...,7452.28,4987.27,12439.55,56558.73,123789.1177,37100.54,23,80069.28,4704.679,0.0448


[
  {
    "index": 0,
    "Scenario": "BAU",
    "PV Size (kW)": "-",
    "Battery Size (kW)": "-",
    "Battery Capacity (kWh)": "-",
    "PV Total Electricity Produced (kWh)": "-",
    "Grid Purchased Electricity (kWh)": 100000.0,
    "Electricity Energy Cost ($)": 9893.91,
    "Electricity Demand Cost ($)": 7689.5,
    "Utility Fixed Cost($)": 1600.0,
    "Payback Period (years)": "-",
    "Gross Capital Cost ($)": "-",
    "CO2 Emissions (tonnes)": 4.2,
    "Year 1 Utility Savings (%)": "-",
    "RE Penetration (%)": "-",
    "Annual CO2 Emissions (Tons)": 4.2,
    "Lifecycle CO2 Emissions (Tons)": 84.0,
    "Lifecycle CO2 Reduction (%)": "-",
    "Year 1 Electric Grid Purchases (kWh)": 100000.0,
    "Year 1 Energy Charges ($)": 9893.91,
    "Year 1 Demand Charges ($)": 7689.5,
    "Year 1 Fixed Cost Charges ($)": 1600.0,
    "Year 1 Total Electric Bill Costs ($)": 19183.41,
    "Year 1 Energy Charge Savings ($)": "-",
    "Year 1 Demand Charge Savings ($)": "-",
    "Year 1 Total 

### Old Script that creates the csv directly without using JSON formatting.

In [ ]:
# def main():
#     run_uuid_1 = "0ea46c21-0076-4437-98b9-f1d3f4d5255d"
#     run_uuid_2 = "c9835dac-7ff7-43b1-9676-6bd135e3d167"

#     run_uuids = [run_uuid_1, run_uuid_2]

#     scenarios = summary_by_runuuids(run_uuids)
#     if 'scenarios' not in scenarios:
#         print(f"Error: {scenarios['Error']}")
#         return

#     final_df = process_scenarios(scenarios['scenarios'])
#     final_df = remove_empty_data_columns(final_df)

#     final_df.iloc[1:, 0] = run_uuids
#     display(final_df)
    
#     # Save transposed DataFrame as CSV
#     transposed_df = final_df.T
#     transposed_df.to_csv('transposed_results.csv')

# main()